# VibeCurator - Stage 1: Item2Vec Baseline

이 노트북은 Melon Playlist Dataset의 train.json을 사용하여 **item2vec(Word2Vec-SGNS)** 방식으로 곡 임베딩을 학습합니다.

각 플레이리스트의 곡 시퀀스를 "문장", 곡 ID를 "단어"로 취급하여 CF(Collaborative Filtering) 기반 곡 추천을 구현합니다.

In [ ]:
# Step 0: 기본 설정
# 필수 라이브러리 import 및 하이퍼파라미터 정의

import json
import os
import logging
from multiprocessing import cpu_count
import numpy as np
from gensim.models import Word2Vec
import pandas as pd  # 간단 확인용

# 로깅 기본 설정
logging.basicConfig(level=logging.INFO, format='%(message)s')
logger = logging.getLogger(__name__)

# ========================================
# 데이터 경로 설정 (recommend_model 폴더 기준)
# ========================================
DATA_ROOT = "../melon-dataset-excepttar"  # recommend_model 폴더 기준
TRAIN_JSON_PATH = os.path.join(DATA_ROOT, "train.json")

# 모델 저장 경로
MODEL_DIR = "../models"  # recommend_model 폴더 기준
os.makedirs(MODEL_DIR, exist_ok=True)

# ========================================
# item2vec 하이퍼파라미터 설정
# ========================================
HYPERPARAMS = {
    "EMBEDDING_DIM": 128,      # 임베딩 차원
    "WINDOW_SIZE": 10,         # 컨텍스트 윈도우 크기 (플레이리스트 내 곡 간격)
    "MIN_COUNT": 2,            # 최소 출현 횟수 (2회 미만 곡은 제외)
    "NEGATIVE": 10,            # Negative sampling 개수
    "EPOCHS": 5,               # 학습 에폭 수
    "SG": 1,                   # Skip-gram (1) or CBOW (0)
    "WORKERS": cpu_count()     # 멀티프로세싱 워커 수
}

logger.info("✅ Step 0 완료: 라이브러리 및 하이퍼파라미터 설정 완료")
logger.info(f"   - 임베딩 차원: {HYPERPARAMS['EMBEDDING_DIM']}")
logger.info(f"   - 윈도우 크기: {HYPERPARAMS['WINDOW_SIZE']}")
logger.info(f"   - 학습 에폭: {HYPERPARAMS['EPOCHS']}")

✅ Step 0 완료: 라이브러리 및 하이퍼파라미터 설정 완료
   - 임베딩 차원: 128
   - 윈도우 크기: 10
   - 학습 에폭: 5


In [13]:
# Step 1: 데이터 로드 & 기본 통계
# train.json을 로드하고 플레이리스트 및 곡 기본 정보를 확인합니다

logger.info("📂 Step 1 시작: train.json 로드 중...")

# train.json 로드
# TODO: 대용량 데이터의 경우 메모리 이슈가 있을 수 있으니, generator 방식으로 변경 고려
with open(TRAIN_JSON_PATH, "r", encoding="utf-8") as f:
    train_data = json.load(f)

logger.info(f"✅ train.json 로드 완료")

# ========================================
# 기본 통계 확인
# ========================================
num_playlists = len(train_data)
logger.info(f"\n📊 기본 통계:")
logger.info(f"   - 총 플레이리스트 수: {num_playlists:,}")

# 첫 번째 플레이리스트 구조 확인
sample_playlist = train_data[0]
logger.info(f"\n📝 첫 번째 플레이리스트 구조:")
logger.info(f"   - Keys: {list(sample_playlist.keys())}")
logger.info(f"   - 곡 수: {len(sample_playlist.get('songs', []))}")
logger.info(f"   - 예시 song_id: {sample_playlist.get('songs', [])[:5]}")

# 고유 곡 ID 개수 및 플레이리스트 길이 계산
all_song_ids = set()
playlist_lengths = []

for playlist in train_data:
    songs = playlist.get('songs', [])
    all_song_ids.update(songs)
    playlist_lengths.append(len(songs))

logger.info(f"\n🎵 곡 통계:")
logger.info(f"   - 고유 곡 ID 개수: {len(all_song_ids):,}")

# ========================================
# 플레이리스트 길이 상세 통계 (Window Size 조정용)
# ========================================
logger.info(f"\n📏 플레이리스트 길이 분포 (Window Size 참고용):")
logger.info(f"   - 최소값: {min(playlist_lengths)}곡")
logger.info(f"   - 25% 백분위: {np.percentile(playlist_lengths, 25):.0f}곡")
logger.info(f"   - 50% 백분위 (중간값): {np.percentile(playlist_lengths, 50):.0f}곡")
logger.info(f"   - 75% 백분위: {np.percentile(playlist_lengths, 75):.0f}곡")
logger.info(f"   - 90% 백분위: {np.percentile(playlist_lengths, 90):.0f}곡")
logger.info(f"   - 95% 백분위: {np.percentile(playlist_lengths, 95):.0f}곡")
logger.info(f"   - 최대값: {max(playlist_lengths)}곡")
logger.info(f"   - 평균: {np.mean(playlist_lengths):.1f}곡")
logger.info(f"   - 표준편차: {np.std(playlist_lengths):.1f}곡")

# 길이별 플레이리스트 개수 분포
logger.info(f"\n📊 플레이리스트 길이 구간별 분포:")
bins = [0, 10, 20, 30, 50, 100, float('inf')]
labels = ['1-10곡', '11-20곡', '21-30곡', '31-50곡', '51-100곡', '100곡 초과']
hist, _ = np.histogram(playlist_lengths, bins=bins)

for label, count in zip(labels, hist):
    percentage = (count / len(playlist_lengths)) * 100
    logger.info(f"   - {label:12s}: {count:6,}개 ({percentage:5.1f}%)")

# Window Size 추천
median_len = np.median(playlist_lengths)
recommended_window = int(median_len / 2)
logger.info(f"\n💡 Window Size 추천:")
logger.info(f"   - 중간값의 절반 기준: {recommended_window}")
logger.info(f"   - 보수적 설정 (짧은 플리 고려): 5-10")
logger.info(f"   - 적극적 설정 (긴 플리 활용): 10-20")
logger.info(f"   - 현재 설정값: {HYPERPARAMS['WINDOW_SIZE']}")

logger.info("\n✅ Step 1 완료: 데이터 로드 및 기본 통계 확인 완료")

📂 Step 1 시작: train.json 로드 중...
✅ train.json 로드 완료

📊 기본 통계:
   - 총 플레이리스트 수: 115,071

📝 첫 번째 플레이리스트 구조:
   - Keys: ['tags', 'id', 'plylst_title', 'songs', 'like_cnt', 'updt_date']
   - 곡 수: 19
   - 예시 song_id: [525514, 129701, 383374, 562083, 297861]

🎵 곡 통계:
   - 고유 곡 ID 개수: 615,142

📏 플레이리스트 길이 분포 (Window Size 참고용):
   - 최소값: 1곡
   - 25% 백분위: 19곡
   - 50% 백분위 (중간값): 30곡
   - 75% 백분위: 54곡
   - 90% 백분위: 100곡
   - 95% 백분위: 156곡
   - 최대값: 200곡
   - 평균: 45.9곡
   - 표준편차: 44.0곡

📊 플레이리스트 길이 구간별 분포:
   - 1-10곡       :  3,164개 (  2.7%)
   - 11-20곡      : 26,753개 ( 23.2%)
   - 21-30곡      : 27,500개 ( 23.9%)
   - 31-50곡      : 25,372개 ( 22.0%)
   - 51-100곡     : 20,523개 ( 17.8%)
   - 100곡 초과     : 11,759개 ( 10.2%)

💡 Window Size 추천:
   - 중간값의 절반 기준: 15
   - 보수적 설정 (짧은 플리 고려): 5-10
   - 적극적 설정 (긴 플리 활용): 10-20
   - 현재 설정값: 10

✅ Step 1 완료: 데이터 로드 및 기본 통계 확인 완료


In [19]:
# Step 2: 시퀀스 생성용 클래스 정의
# PlaylistSentenceIterator는 플레이리스트를 "문장(곡 ID 시퀀스)"으로 변환하는 iterator입니다

class PlaylistSentenceIterator:
    """
    Melon 플레이리스트를 gensim Word2Vec이 이해할 수 있는 문장(토큰 리스트)으로 변환하는 iterator
    
    Args:
        playlists: train.json에서 로드한 플레이리스트 리스트
        min_len: 최소 곡 개수 (이보다 짧은 플레이리스트는 제외)
    
    Yields:
        List[str]: 곡 ID를 문자열로 변환한 토큰 리스트
    """
    
    def __init__(self, playlists, min_len=2):
        self.playlists = playlists
        self.min_len = min_len
        logger.info(f"🔄 PlaylistSentenceIterator 초기화: 최소 곡 수 = {min_len}")
    
    def __iter__(self):
        for playlist in self.playlists:
            songs = playlist.get('songs', [])
            
            # 곡 수가 최소 길이보다 적으면 건너뛰기
            if len(songs) < self.min_len:
                continue
            
            # 곡 ID를 문자열로 변환하여 yield
            # NOTE: gensim Word2Vec은 문자열 토큰을 받습니다
            yield [str(song_id) for song_id in songs]

# ========================================
# 간단한 테스트
# ========================================
logger.info("\n🧪 PlaylistSentenceIterator 테스트...")

iterator = PlaylistSentenceIterator(train_data, min_len=2)
iterator_test = iter(iterator)
#iter(obj) → obj.__iter__()를 호출 : iter 라는 내장함수

# 처음 3개 문장 샘플 출력
# next()로 iterator_test를 호출 , iterator_test = iter(iterator) 니깐 iter 함수를 호출
# iter 함수에서 song_id를 word2vec모델이 받을 수있게 str로 형변환
# yield 문으로 리턴 하므로 한번에 하나씩 반환 가능
# rage(3) 즉 세번동안 next()로 반복해서 클래스가 잘 작동하나 확인
for i in range(3):
    sentence = next(iterator_test)
    logger.info(f"   - 문장 {i+1}: {len(sentence)}개 토큰, 예시: {sentence[:5]}")

logger.info("\n✅ Step 2 완료: PlaylistSentenceIterator 정의 및 테스트 완료")

2025-11-26 21:15:17,048 - INFO - 
🧪 PlaylistSentenceIterator 테스트...
2025-11-26 21:15:17,049 - INFO - 🔄 PlaylistSentenceIterator 초기화: 최소 곡 수 = 2
2025-11-26 21:15:17,087 - INFO -    - 문장 1: 19개 토큰, 예시: ['525514', '129701', '383374', '562083', '297861']
2025-11-26 21:15:17,088 - INFO -    - 문장 2: 42개 토큰, 예시: ['432406', '675945', '497066', '120377', '389529']
2025-11-26 21:15:17,088 - INFO -    - 문장 3: 28개 토큰, 예시: ['83116', '276692', '166267', '186301', '354465']
2025-11-26 21:15:17,089 - INFO - 
✅ Step 2 완료: PlaylistSentenceIterator 정의 및 테스트 완료


In [20]:
# Step 3: item2vec 학습 함수 정의
# Word2Vec-SGNS 방식으로 곡 임베딩을 학습하는 함수를 정의합니다

def train_item2vec(sentences, hyperparams):
    """
    gensim Word2Vec을 사용하여 item2vec 모델을 학습합니다.
    
    Args:
        sentences: PlaylistSentenceIterator 또는 토큰 리스트의 리스트
        hyperparams: 하이퍼파라미터 딕셔너리
    
    Returns:
        gensim.models.Word2Vec: 학습된 Word2Vec 모델
    """
    logger.info("\n🚀 item2vec 학습 시작...")
    logger.info(f"   - 임베딩 차원: {hyperparams['EMBEDDING_DIM']}")
    logger.info(f"   - 윈도우 크기: {hyperparams['WINDOW_SIZE']}")
    logger.info(f"   - 최소 출현 횟수: {hyperparams['MIN_COUNT']}")
    logger.info(f"   - Negative sampling: {hyperparams['NEGATIVE']}")
    logger.info(f"   - 에폭: {hyperparams['EPOCHS']}")
    
    # NOTE: iterator를 두 번 사용하기 위해 리스트로 캐싱
    # TODO: 메모리가 부족한 경우, corpus_file 옵션을 사용한 디스크 기반 학습 고려
    logger.info("   - 문장 리스트 캐싱 중... (메모리 사용 주의)")
    sentences_list = list(sentences)
    logger.info(f"   - 총 {len(sentences_list):,}개 플레이리스트 준비 완료")
    
    # Word2Vec 모델 생성
    model = Word2Vec(
        vector_size=hyperparams['EMBEDDING_DIM'],
        window=hyperparams['WINDOW_SIZE'],
        min_count=hyperparams['MIN_COUNT'],
        sg=hyperparams['SG'],  # Skip-gram
        negative=hyperparams['NEGATIVE'],
        workers=hyperparams['WORKERS'],
        epochs=hyperparams['EPOCHS'],
        seed=42  # 재현성을 위한 시드
    )
    
    # Vocabulary 구축
    logger.info("\n📚 Vocabulary 구축 중...")
    model.build_vocab(sentences_list, progress_per=10000)
    logger.info(f"   - Vocabulary 크기: {len(model.wv):,}개 곡")
    
    # 모델 학습
    logger.info("\n🔥 모델 학습 중...")
    model.train(
        sentences_list,
        total_examples=model.corpus_count,
        epochs=model.epochs,
        report_delay=1.0
    )
    
    logger.info("\n✅ item2vec 학습 완료!")
    logger.info(f"   - 최종 Vocabulary 크기: {len(model.wv):,}개 곡")
    
    return model

logger.info("✅ Step 3 완료: train_item2vec 함수 정의 완료")

2025-11-26 21:15:20,386 - INFO - ✅ Step 3 완료: train_item2vec 함수 정의 완료


In [21]:
# Step 4: item2vec 학습 실행
# 실제로 모델을 학습시킵니다 (시간이 다소 걸릴 수 있습니다)

logger.info("\n" + "="*60)
logger.info("Step 4: item2vec 모델 학습 실행")
logger.info("="*60)

# PlaylistSentenceIterator 생성
sentences = PlaylistSentenceIterator(train_data, min_len=2)

# item2vec 학습 실행
w2v_model = train_item2vec(sentences, HYPERPARAMS)

# ========================================
# 학습 결과 확인
# ========================================
logger.info("\n📊 학습 결과 요약:")
logger.info(f"   - Vocabulary 크기: {len(w2v_model.wv):,}개 곡")
logger.info(f"   - 임베딩 차원: {w2v_model.wv.vector_size}")

# 예시 곡 벡터 확인
sample_songs = list(w2v_model.wv.index_to_key)[:3]
logger.info("\n🎵 예시 곡 임베딩:")
for song_id in sample_songs:
    vector = w2v_model.wv[song_id]
    logger.info(f"   - Song ID {song_id}: shape={vector.shape}, norm={np.linalg.norm(vector):.4f}")

logger.info("\n✅ Step 4 완료: 모델 학습 실행 완료")

2025-11-26 21:15:24,960 - INFO - 
2025-11-26 21:15:24,961 - INFO - Step 4: item2vec 모델 학습 실행
2025-11-26 21:15:24,962 - INFO - ============================================================
2025-11-26 21:15:24,963 - INFO - 🔄 PlaylistSentenceIterator 초기화: 최소 곡 수 = 2
2025-11-26 21:15:24,964 - INFO - 
🚀 item2vec 학습 시작...
2025-11-26 21:15:24,964 - INFO -    - 임베딩 차원: 128
2025-11-26 21:15:24,965 - INFO -    - 윈도우 크기: 10
2025-11-26 21:15:24,965 - INFO -    - 최소 출현 횟수: 2
2025-11-26 21:15:24,966 - INFO -    - Negative sampling: 10
2025-11-26 21:15:24,966 - INFO -    - 에폭: 5
2025-11-26 21:15:24,966 - INFO -    - 문장 리스트 캐싱 중... (메모리 사용 주의)
2025-11-26 21:15:34,555 - INFO -    - 총 115,040개 플레이리스트 준비 완료
2025-11-26 21:15:34,586 - INFO - Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=128, alpha=0.025>', 'datetime': '2025-11-26T21:15:34.584785', 'gensim': '4.4.0', 'python': '3.13.7 | packaged by Anaconda, Inc. | (main, Sep  9 2025, 19:54:37) [MSC v.1929 64 bit (AMD64)]', 'platform': '

In [22]:
# Step 5: 모델 저장/로드
# 학습된 모델을 저장하고, 로드하는 방법을 확인합니다

logger.info("\n" + "="*60)
logger.info("Step 5: 모델 저장 및 로드")
logger.info("="*60)

# ========================================
# 모델 저장
# ========================================
model_path = os.path.join(MODEL_DIR, "v2_item2vec.model")
logger.info(f"\n💾 모델 저장 중: {model_path}")

w2v_model.save(model_path)
logger.info("✅ 모델 저장 완료")

# ========================================
# 모델 로드 테스트
# ========================================
logger.info(f"\n📂 모델 로드 테스트: {model_path}")
loaded_model = Word2Vec.load(model_path)
logger.info("✅ 모델 로드 완료")

# 로드된 모델 검증
logger.info("\n🔍 로드된 모델 검증:")
logger.info(f"   - Vocabulary 크기: {len(loaded_model.wv):,}개 곡")
logger.info(f"   - 임베딩 차원: {loaded_model.wv.vector_size}")

# 원본 모델과 로드된 모델의 벡터 비교
test_song_id = list(w2v_model.wv.index_to_key)[0]
original_vector = w2v_model.wv[test_song_id]
loaded_vector = loaded_model.wv[test_song_id]

vector_diff = np.linalg.norm(original_vector - loaded_vector)
logger.info(f"\n✔️  벡터 일치 확인 (차이 norm): {vector_diff:.10f}")
if vector_diff < 1e-6:
    logger.info("   ✅ 저장/로드가 정상적으로 동작합니다!")
else:
    logger.warning("   ⚠️  벡터 차이가 감지되었습니다.")

logger.info("\n✅ Step 5 완료: 모델 저장/로드 테스트 완료")

2025-11-26 21:42:24,326 - INFO - 
2025-11-26 21:42:24,328 - INFO - Step 5: 모델 저장 및 로드
2025-11-26 21:42:24,329 - INFO - ============================================================
2025-11-26 21:42:24,331 - INFO - 
💾 모델 저장 중: ./models\v2_item2vec.model
2025-11-26 21:42:24,339 - INFO - Word2Vec lifecycle event {'fname_or_handle': './models\\v2_item2vec.model', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2025-11-26T21:42:24.339164', 'gensim': '4.4.0', 'python': '3.13.7 | packaged by Anaconda, Inc. | (main, Sep  9 2025, 19:54:37) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'saving'}
2025-11-26 21:42:24,353 - INFO - storing np array 'vectors' to ./models\v2_item2vec.model.wv.vectors.npy
2025-11-26 21:42:24,822 - INFO - storing np array 'syn1neg' to ./models\v2_item2vec.model.syn1neg.npy
2025-11-26 21:42:25,662 - INFO - not storing attribute cum_table
2025-11-26 21:42:26,516 - INFO - saved ./models\v2_item2vec.model
202

In [14]:
# Step 6: 유사 곡 추천 함수 + 예제
# 학습된 item2vec 모델을 사용하여 유사한 곡을 추천하는 함수를 정의합니다

# =========================================================
# (추가) 저장된 모델 로드 로직
# 커널 재시작 후 Step 4(학습)를 건너뛰고 실행할 경우를 위해 모델을 로드합니다.
# =========================================================
if 'w2v_model' not in locals():
    try:
        # Step 0을 실행했다면 MODEL_DIR, Word2Vec 등이 정의되어 있어야 함
        # 만약 정의되지 않았다면 임시로 import 및 설정
        if 'Word2Vec' not in locals():
            from gensim.models import Word2Vec
        if 'os' not in locals():
            import os
        if 'MODEL_DIR' not in locals():
            MODEL_DIR = "./models"
            
        _model_path = "C:/Users/ASUS/music_recommend/work/models/models/v2_item2vec.model"
        # os.path.join(MODEL_DIR, "v2_item2vec.model")
        
        if os.path.exists(_model_path):
            if 'logger' in locals():
                logger.info(f"🔄 저장된 모델을 로드합니다: {_model_path}")
            else:
                print(f"🔄 저장된 모델을 로드합니다: {_model_path}")
                
            w2v_model = Word2Vec.load(_model_path)
            
            if 'logger' in locals():
                logger.info("✅ 모델 로드 완료")
            else:
                print("✅ 모델 로드 완료")
        else:
            msg = f"❌ 모델 파일을 찾을 수 없습니다: {_model_path}"
            if 'logger' in locals(): logger.warning(msg)
            else: print(msg)
            
    except Exception as e:
        msg = f"⚠️ 모델 로드 중 오류 발생: {e}"
        if 'logger' in locals(): logger.error(msg)
        else: print(msg)

def get_similar_songs(model, song_id, topn=20):
    """
    주어진 곡 ID와 유사한 곡들을 추천합니다.
    
    Args:
        model: 학습된 Word2Vec 모델
        song_id: 시드 곡 ID (int 또는 str)
        topn: 추천할 곡 개수
    
    Returns:
        List[Tuple[int, float]]: [(song_id, similarity_score), ...] 형태의 리스트
    """
    # song_id를 문자열로 변환 (모델에서 사용하는 토큰 형식)
    song_id_str = str(song_id)
    
    # 모델에 해당 곡이 없는 경우
    if song_id_str not in model.wv:
        logger.warning(f"⚠️  곡 ID '{song_id}'가 모델에 없습니다. (최소 출현 횟수 미달 가능성)")
        return []
    
    # gensim의 most_similar 사용
    similar_items = model.wv.most_similar(song_id_str, topn=topn)
    
    # (str_id, similarity) -> (int_id, similarity) 변환
    results = [(int(item_id), similarity) for item_id, similarity in similar_items]
    
    return results

logger.info("\n" + "="*60)
logger.info("Step 6: 유사 곡 추천 함수 정의 및 테스트")
logger.info("="*60)

# ========================================
# 추천 함수 테스트
# ========================================
# train.json에서 적당한 seed_song 선택 (첫 번째 플레이리스트의 첫 곡)
if 'train_data' in locals():
    seed_song_id = train_data[0]['songs'][0]
elif 'w2v_model' in locals():
    seed_song_id = int(w2v_model.wv.index_to_key[0])
    if 'logger' in locals(): logger.info("⚠️ train_data가 메모리에 없어 모델의 첫 번째 곡을 시드로 사용합니다.")
else:
    seed_song_id = 0

logger.info(f"\n🎯 시드 곡 ID: {seed_song_id}")

# 유사 곡 추천
similar_songs = get_similar_songs(w2v_model, seed_song_id, topn=10)

if similar_songs:
    logger.info(f"\n🎵 추천 결과 (Top 10):")
    logger.info("   순위 | Song ID    | 유사도")
    logger.info("   " + "-"*35)
    for rank, (song_id, similarity) in enumerate(similar_songs, 1):
        logger.info(f"   {rank:2d}   | {song_id:10d} | {similarity:.4f}")
else:
    logger.warning("   추천 결과가 없습니다.")

# TODO: 추후 song_meta.json을 활용하여 곡명, 아티스트명 등 메타데이터를 함께 표시 예정

logger.info("\n✅ Step 6 완료: 유사 곡 추천 함수 정의 및 테스트 완료")


Step 6: 유사 곡 추천 함수 정의 및 테스트

🎯 시드 곡 ID: 525514

🎵 추천 결과 (Top 10):
   순위 | Song ID    | 유사도
   -----------------------------------
    1   |     695342 | 0.9845
    2   |     658448 | 0.9788
    3   |     423887 | 0.9784
    4   |      15811 | 0.9753
    5   |     552048 | 0.9753
    6   |     108392 | 0.9738
    7   |     650362 | 0.9736
    8   |     487399 | 0.9734
    9   |     200922 | 0.9732
   10   |     343991 | 0.9730

✅ Step 6 완료: 유사 곡 추천 함수 정의 및 테스트 완료


In [23]:
# Step 7 (선택): 간단한 평가/체감 테스트
# item2vec이 얼마나 합리적인 추천을 하는지 대략적으로 확인합니다

logger.info("\n" + "="*60)
logger.info("Step 7 (선택): 평가 및 체감 테스트")
logger.info("="*60)

# ========================================
# 테스트 1: 같은 플레이리스트 내 곡 추천 재현율
# ========================================
logger.info("\n🧪 테스트 1: 플레이리스트 내 곡 추천 재현율 확인")
logger.info("   (같은 플레이리스트에 있던 곡이 추천 결과에 얼마나 포함되는지 확인)\n")

# 테스트용 플레이리스트 선택 (곡이 충분히 많은 것)
if 'train_data' in locals():
    test_playlists = [p for p in train_data if len(p.get('songs', [])) >= 10][:5]
else:
    if 'logger' in locals(): logger.warning("⚠️ train_data가 메모리에 없습니다. Step 1을 실행해주세요. (테스트 1 건너뜀)")
    test_playlists = []

recall_scores = []

for idx, playlist in enumerate(test_playlists, 1):
    songs = playlist['songs']
    seed_song = songs[0]  # 첫 번째 곡을 시드로 사용
    ground_truth = set(songs[1:])  # 나머지 곡들이 정답 후보
    
    # 추천 결과
    recommendations = get_similar_songs(w2v_model, seed_song, topn=20)
    recommended_ids = set([song_id for song_id, _ in recommendations])
    
    # 재현율 계산 (추천된 곡 중 실제 플레이리스트에 있던 곡의 비율)
    hits = recommended_ids & ground_truth
    recall = len(hits) / len(ground_truth) if len(ground_truth) > 0 else 0
    recall_scores.append(recall)
    
    logger.info(f"   플레이리스트 {idx}:")
    logger.info(f"      - 시드 곡: {seed_song}")
    logger.info(f"      - 정답 곡 수: {len(ground_truth)}")
    logger.info(f"      - 추천 곡 중 정답: {len(hits)}개")
    logger.info(f"      - 재현율: {recall:.2%}\n")

avg_recall = np.mean(recall_scores)
logger.info(f"📊 평균 재현율 (Top 20): {avg_recall:.2%}")

# ========================================
# 테스트 2: 다양한 시드 곡으로 추천 결과 확인
# ========================================
logger.info("\n🧪 테스트 2: 다양한 시드 곡 추천 결과 샘플링\n")

# 랜덤하게 몇 개 곡 선택
np.random.seed(42)
vocab_keys = list(w2v_model.wv.index_to_key)
sample_seeds = np.random.choice(vocab_keys, size=min(3, len(vocab_keys)), replace=False)

for seed in sample_seeds:
    logger.info(f"🎯 시드 곡 ID: {seed}")
    recommendations = get_similar_songs(w2v_model, int(seed), topn=5)
    
    if recommendations:
        logger.info("   추천 결과:")
        for rank, (song_id, similarity) in enumerate(recommendations, 1):
            logger.info(f"      {rank}. Song ID {song_id} (유사도: {similarity:.4f})")
    logger.info("")

logger.info("\n" + "="*60)
logger.info("✅ Step 7 완료: 평가 및 체감 테스트 완료")
logger.info("="*60)
logger.info("\n🎉 Stage 1 item2vec 베이스라인 구축이 완료되었습니다!")
logger.info("\n💡 다음 단계:")
logger.info("   - song_meta.json을 활용한 메타데이터 통합")
logger.info("   - 태그/플레이리스트 제목을 함께 학습하는 확장 버전")
logger.info("   - Stage 2: 메타데이터 기반 Rule Refinement")

2025-11-26 21:43:50,686 - INFO - 
2025-11-26 21:43:50,688 - INFO - Step 7 (선택): 평가 및 체감 테스트
2025-11-26 21:43:50,689 - INFO - ============================================================
2025-11-26 21:43:50,690 - INFO - 
🧪 테스트 1: 플레이리스트 내 곡 추천 재현율 확인
2025-11-26 21:43:50,691 - INFO -    (같은 플레이리스트에 있던 곡이 추천 결과에 얼마나 포함되는지 확인)

2025-11-26 21:43:51,009 - INFO -    플레이리스트 1:
2025-11-26 21:43:51,009 - INFO -       - 시드 곡: 525514
2025-11-26 21:43:51,010 - INFO -       - 정답 곡 수: 18
2025-11-26 21:43:51,010 - INFO -       - 추천 곡 중 정답: 1개
2025-11-26 21:43:51,011 - INFO -       - 재현율: 5.56%

2025-11-26 21:43:51,021 - INFO -    플레이리스트 2:
2025-11-26 21:43:51,022 - INFO -       - 시드 곡: 432406
2025-11-26 21:43:51,023 - INFO -       - 정답 곡 수: 41
2025-11-26 21:43:51,023 - INFO -       - 추천 곡 중 정답: 0개
2025-11-26 21:43:51,023 - INFO -       - 재현율: 0.00%

2025-11-26 21:43:51,034 - INFO -    플레이리스트 3:
2025-11-26 21:43:51,035 - INFO -       - 시드 곡: 83116
2025-11-26 21:43:51,035 - INFO -       - 정답 곡 수: 27
202

In [15]:
# Step 7.5: 전체 데이터셋 Recall@20 평가 (Simple Version)
# 불필요한 로그를 제거하고 핵심 지표(Recall@20)만 출력합니다.

logger.info("\n" + "="*60)
logger.info("Step 7.5: 전체 데이터셋 Recall@20 평가")
logger.info("="*60)

if 'train_data' in locals():
    # 1. 평가 대상 선정 (곡 10개 이상)
    valid_playlists = [p for p in train_data if len(p.get('songs', [])) >= 10]
    
    # 2. 샘플링 (1,000개)
    import random
    random.seed(42)
    sample_size = min(1000, len(valid_playlists))
    eval_playlists = random.sample(valid_playlists, sample_size)
    
    logger.info(f"🔍 평가 진행 중... (샘플 {sample_size}개)")
    
    recall_scores = []
    missing_seed_count = 0
    
    # 3. 평가 루프
    for playlist in eval_playlists:
        songs = playlist['songs']
        seed_song = songs[0]
        ground_truth = set(songs[1:])
        
        # 시드 곡이 모델에 있는지 확인 (로그 없이 조용히 처리)
        if str(seed_song) not in w2v_model.wv:
            missing_seed_count += 1
            recall_scores.append(0) # 추천 불가하므로 Recall 0
            continue
            
        # 추천 (경고 로그를 피하기 위해 직접 모델 호출)
        try:
            similar_items = w2v_model.wv.most_similar(str(seed_song), topn=20)
            recommended_ids = set([int(sid) for sid, _ in similar_items])
            
            # Recall 계산
            hits = recommended_ids & ground_truth
            recall = len(hits) / len(ground_truth) if len(ground_truth) > 0 else 0
            recall_scores.append(recall)
            
        except KeyError:
            missing_seed_count += 1
            recall_scores.append(0)
            
    # 4. 결과 출력
    avg_recall = np.mean(recall_scores)
    
    logger.info(f"\n📊 최종 평가 결과:")
    logger.info(f"   ► Average Recall@20 : {avg_recall:.4f} ({avg_recall*100:.2f}%)")
    logger.info(f"   ► (참고) 모델 미포함 시드 곡 : {missing_seed_count}개 ({missing_seed_count/sample_size*100:.1f}%)")
    logger.info("="*60)

else:
    logger.warning("⚠️ train_data가 메모리에 없습니다.")


Step 7.5: 전체 데이터셋 Recall@20 평가
🔍 평가 진행 중... (샘플 1000개)

📊 최종 평가 결과:
   ► Average Recall@20 : 0.0253 (2.53%)
   ► (참고) 모델 미포함 시드 곡 : 49개 (4.9%)


In [25]:
# Step 8 (추가): 메타데이터 포함 추천 결과 확인
# song_meta.json을 로드하여 곡명, 아티스트 정보를 함께 보여줍니다

import os
import json
import pandas as pd
from IPython.display import display

# 1. song_meta.json 로드
SONG_META_JSON_PATH = os.path.join(DATA_ROOT, "song_meta.json")

if os.path.exists(SONG_META_JSON_PATH):
    logger.info("📂 song_meta.json 로드 중...")
    with open(SONG_META_JSON_PATH, "r", encoding="utf-8") as f:
        song_meta = json.load(f)
    
    # 검색 속도를 위해 딕셔너리로 변환 (List -> Dict)
    # key: song_id, value: song_info
    song_meta_dict = {song['id']: song for song in song_meta}
    logger.info(f"✅ 메타데이터 로드 완료: {len(song_meta_dict):,}곡")

    # 2. 메타데이터 포함 추천 함수 정의
    def show_recommendations_with_meta(seed_id, topn=10):
        # 유사 곡 추천 받기
        recs = get_similar_songs(w2v_model, seed_id, topn=topn)
        
        rows = []
        for sid, score in recs:
            # 메타데이터 조회
            meta = song_meta_dict.get(sid)
            
            if meta:
                song_name = meta.get("song_name", "Unknown")
                # 아티스트는 리스트 형태(artist_name_basket)로 제공됨
                artists = meta.get("artist_name_basket", [])
                artist_str = ", ".join(artists) if artists else "Unknown"
                genres = meta.get("song_gn_gnr_basket", [])
            else:
                song_name = f"Unknown (ID: {sid})"
                artist_str = "-"
                genres = []

            rows.append({
                "Song ID": sid,
                "유사도": score,
                "곡명": song_name,
                "아티스트": artist_str,
                "장르": genres
            })
            
        return pd.DataFrame(rows)

    # 3. 함수 실행 및 결과 출력
    # 예시: 학습 데이터에 있는 첫 번째 곡으로 테스트
    # (Step 6에서 사용했던 seed_song_id 재사용)
    if 'seed_song_id' not in locals():
        seed_song_id = train_data[0]['songs'][0]
        
    # 시드 곡 정보 확인
    seed_meta = song_meta_dict.get(seed_song_id)
    if seed_meta:
        seed_name = seed_meta.get("song_name", "Unknown")
        seed_artists = seed_meta.get("artist_name_basket", [])
        seed_artist_str = ", ".join(seed_artists) if seed_artists else "Unknown"
    else:
        seed_name = "Unknown"
        seed_artist_str = "Unknown"

    logger.info(f"\n🎯 시드 곡 정보: [{seed_name}] - {seed_artist_str} (ID: {seed_song_id})")
    logger.info("   추천 결과 (메타데이터 포함):")
    
    # DataFrame 생성 및 출력
    rec_df = show_recommendations_with_meta(seed_song_id)
    display(rec_df)

else:
    logger.warning(f"⚠️ song_meta.json 파일을 찾을 수 없습니다: {SONG_META_JSON_PATH}")
    logger.warning("DATA_ROOT 경로를 확인하거나 파일을 업로드해주세요.")

2025-11-26 21:44:32,810 - INFO - 📂 song_meta.json 로드 중...
2025-11-26 21:44:40,077 - INFO - ✅ 메타데이터 로드 완료: 707,989곡
2025-11-26 21:44:40,078 - INFO - 
🎯 시드 곡 정보: [Hey Little Girl] - The Sol (ID: 525514)
2025-11-26 21:44:40,078 - INFO -    추천 결과 (메타데이터 포함):


,Song ID,유사도,곡명,아티스트,장르
0,695342,0.984490,I Want Crazy (Encore),Hunter Hayes,[GN1400]
1,658448,0.978754,Moving,Travis,[GN1000]
2,423887,0.978368,Nothing Stays The Same,Luke Sital-Singh,[GN1400]
3,15811,0.975316,I Wish I Could Break Your Heart,Cassadee Pope,[GN1400]
4,552048,0.975251,We Don`t Have To Take Our Clothes Off,Ella Eyre,[GN1300]
5,108392,0.973768,Architect,"Frightened Rabbit, Manchester Orchestra","[GN0900, GN1000]"
6,650362,0.973646,"Sad, Sad World",Jamie Cullum,[GN1700]
7,487399,0.973358,Under (Pop Ver.),Alex Hepburn,[GN0900]
8,200922,0.973239,Today&#39;s Not Yesterday,Shane Filan,[GN0900]
9,343991,0.973036,Dancehall,Tribes,[GN1000]


In [27]:
from collections import Counter

# 1. 전체 곡 등장 횟수 카운트
if 'logger' in locals():
    logger.info("📊 곡 등장 횟수 분포 분석 중... (시간이 조금 걸릴 수 있습니다)")
else:
    print("📊 곡 등장 횟수 분포 분석 중... (시간이 조금 걸릴 수 있습니다)")

# 모든 플레이리스트의 곡을 하나의 리스트로 모으기 (메모리 주의: 너무 크면 generator 방식 사용 권장)
# 여기서는 단순하게 루프 돌면서 카운팅
song_counter = Counter()
for playlist in train_data:
    song_counter.update(playlist['songs'])

# 2. 통계 계산
total_unique_songs = len(song_counter)
count_1 = sum(1 for c in song_counter.values() if c == 1)
count_2 = sum(1 for c in song_counter.values() if c == 2)
count_under_3 = count_1 + count_2
count_3_plus = total_unique_songs - count_under_3

# 3. 결과 출력
msg = f"""
📈 곡 등장 빈도 분석 결과
==================================================
  총 고유 곡 수 (Unique Songs): {total_unique_songs:,}개
==================================================
  [세부 분포]
  - 1번만 등장한 곡 : {count_1:7,}개 ({count_1/total_unique_songs*100:.1f}%)
  - 2번만 등장한 곡 : {count_2:7,}개 ({count_2/total_unique_songs*100:.1f}%)
--------------------------------------------------
  ► 3번 미만 (제외됨) : {count_under_3:7,}개 ({count_under_3/total_unique_songs*100:.1f}%)
  ► 3번 이상 (학습됨) : {count_3_plus:7,}개 ({count_3_plus/total_unique_songs*100:.1f}%)
==================================================
"""

if 'logger' in locals():
    logger.info(msg)
else:
    print(msg)

# 4. 간단한 해석
if count_under_3 / total_unique_songs > 0.5:
    print("⚠️ [해석] 전체 곡의 절반 이상이 학습에서 제외되고 있습니다.")
    print("   -> min_count=2로 낮추는 것을 고려해보세요. (Recall 상승 가능성 있음)")
else:
    print("✅ [해석] 제외되는 곡 비율이 양호합니다. min_count=3 유지도 괜찮습니다.")

2025-11-26 21:48:54,271 - INFO - 📊 곡 등장 횟수 분포 분석 중... (시간이 조금 걸릴 수 있습니다)
2025-11-26 21:48:55,706 - INFO - 
📈 곡 등장 빈도 분석 결과
  총 고유 곡 수 (Unique Songs): 615,142개
  [세부 분포]
  - 1번만 등장한 곡 : 299,028개 (48.6%)
  - 2번만 등장한 곡 :  94,458개 (15.4%)
--------------------------------------------------
  ► 3번 미만 (제외됨) : 393,486개 (64.0%)
  ► 3번 이상 (학습됨) : 221,656개 (36.0%)



⚠️ [해석] 전체 곡의 절반 이상이 학습에서 제외되고 있습니다.
   -> min_count=2로 낮추는 것을 고려해보세요. (Recall 상승 가능성 있음)


In [16]:
# [Cell A] 모델 로드 및 Vocab 커버리지 분석

import os
from gensim.models import Word2Vec

# 경로 설정
MODEL_DIR = "./models"
PATH_MIN3 = os.path.join(MODEL_DIR, "models", "v1_item2vec", "item2vec.model")      # 기존 (min=3)
PATH_MIN2 = os.path.join(MODEL_DIR, "models", "v2_item2vec.model") # 신규 (min=2)

logger.info("🔍 모델 로드 및 커버리지 분석 시작...")

# 1. 전체 고유 곡 수 계산 (train_data 기준)
if 'train_data' in locals():
    all_songs = set()
    for p in train_data:
        all_songs.update(p['songs'])
    total_unique_songs = len(all_songs)
else:
    logger.warning("⚠️ train_data가 없습니다. Step 1을 먼저 실행해주세요.")
    total_unique_songs = 615142 # 대략적인 수치 (메타데이터 기준)

logger.info(f"   - 전체 고유 곡 수: {total_unique_songs:,}개")

# 2. 모델 로드 및 Vocab 확인
models = {}

# (1) min_count=3 모델
if os.path.exists(PATH_MIN3):
    model_min3 = Word2Vec.load(PATH_MIN3)
    vocab_min3 = len(model_min3.wv)
    models['min3'] = model_min3
    logger.info(f"   - [min=3] Vocab: {vocab_min3:,}개 (커버리지: {vocab_min3/total_unique_songs*100:.2f}%)")
else:
    logger.warning(f"   - [min=3] 모델 파일 없음: {PATH_MIN3}")

# (2) min_count=2 모델
if os.path.exists(PATH_MIN2):
    model_min2 = Word2Vec.load(PATH_MIN2)
    vocab_min2 = len(model_min2.wv)
    models['min2'] = model_min2
    logger.info(f"   - [min=2] Vocab: {vocab_min2:,}개 (커버리지: {vocab_min2/total_unique_songs*100:.2f}%)")
else:
    logger.warning(f"   - [min=2] 모델 파일 없음: {PATH_MIN2} (학습 필요)")

🔍 모델 로드 및 커버리지 분석 시작...
   - 전체 고유 곡 수: 615,142개
loading Word2Vec object from ./models\models\v1_item2vec\item2vec.model
loading wv recursively from ./models\models\v1_item2vec\item2vec.model.wv.* with mmap=None
loading vectors from ./models\models\v1_item2vec\item2vec.model.wv.vectors.npy with mmap=None
loading syn1neg from ./models\models\v1_item2vec\item2vec.model.syn1neg.npy with mmap=None
setting ignored attribute cum_table to None
Word2Vec lifecycle event {'fname': './models\\models\\v1_item2vec\\item2vec.model', 'datetime': '2025-11-27T17:47:31.285888', 'gensim': '4.4.0', 'python': '3.13.7 | packaged by Anaconda, Inc. | (main, Sep  9 2025, 19:54:37) [MSC v.1929 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19045-SP0', 'event': 'loaded'}
   - [min=3] Vocab: 221,653개 (커버리지: 36.03%)
loading Word2Vec object from ./models\models\v2_item2vec.model
loading wv recursively from ./models\models\v2_item2vec.model.wv.* with mmap=None
loading vectors from ./models\models\v2_item2vec.model.w

In [17]:
# [Cell B] 공통 평가 함수 정의

def evaluate_item2vec_recall(model, playlists, k=20):
    """
    주어진 모델과 플레이리스트 세트에 대해 Recall@K 성능을 평가합니다.
    """
    recall_scores = []
    missing_seed_count = 0
    total_samples = 0
    
    for playlist in playlists:
        songs = playlist.get('songs', [])
        if len(songs) < 2: # 최소 2곡 이상이어야 평가 가능 (Seed + Ground Truth)
            continue
            
        total_samples += 1
        seed_song = str(songs[0])
        ground_truth = set(songs[1:])
        
        # 시드 곡이 모델에 없는 경우 (Cold Start)
        if seed_song not in model.wv:
            missing_seed_count += 1
            recall_scores.append(0.0)
            continue
            
        try:
            # 추천 수행
            similar_items = model.wv.most_similar(seed_song, topn=k)
            recommended_ids = set([int(sid) for sid, _ in similar_items])
            
            # Recall 계산
            hits = recommended_ids & ground_truth
            recall = len(hits) / len(ground_truth) if len(ground_truth) > 0 else 0
            recall_scores.append(recall)
            
        except Exception:
            missing_seed_count += 1
            recall_scores.append(0.0)
            
    # 통계 계산
    if not recall_scores:
        return {
            "recall_mean": 0.0, "recall_median": 0.0, "recall_std": 0.0,
            "cold_start_rate": 0.0, "num_samples": 0
        }
        
    return {
        "recall_mean": np.mean(recall_scores),
        "recall_median": np.median(recall_scores),
        "recall_std": np.std(recall_scores),
        "cold_start_rate": missing_seed_count / total_samples * 100 if total_samples > 0 else 0,
        "num_samples": total_samples
    }

In [18]:
# [Cell C] 성능 비교 실행 (min=3 vs min=2)

if 'train_data' in locals() and len(models) > 0:
    # 1. 평가 샘플링 (공정 비교를 위해 동일 샘플 사용)
    import random
    random.seed(42)
    
    # 곡 10개 이상인 플레이리스트 중 1,000개 추출
    valid_playlists = [p for p in train_data if len(p.get('songs', [])) >= 10]
    sample_size = min(1000, len(valid_playlists))
    playlists_eval = random.sample(valid_playlists, sample_size)
    
    logger.info(f"📊 성능 비교 시작 (샘플 {sample_size}개)...")
    logger.info("="*60)
    
    # 2. 각 모델 평가
    results = {}
    for name, model in models.items():
        res = evaluate_item2vec_recall(model, playlists_eval, k=20)
        results[name] = res
        
        logger.info(f"[{name} 모델 결과]")
        logger.info(f"   ► Recall@20 Mean : {res['recall_mean']:.4f} ({res['recall_mean']*100:.2f}%)")
        logger.info(f"   ► Recall@20 Median: {res['recall_median']:.4f}")
        logger.info(f"   ► Cold Start Rate : {res['cold_start_rate']:.1f}%")
        logger.info("-"*40)
        
    # 3. 최종 비교 요약
    if 'min3' in results and 'min2' in results:
        r3 = results['min3']['recall_mean']
        r2 = results['min2']['recall_mean']
        diff = r2 - r3
        improvement = (diff / r3 * 100) if r3 > 0 else 0
        
        logger.info("📈 [최종 비교]")
        logger.info(f"   min=2 모델이 min=3 대비 Recall이 {improvement:+.1f}% 변화했습니다. ({diff:+.4f})")
        
        if r2 >= r3:
            logger.info("   ✅ min=2 모델 채택 권장 (성능 유지/향상 + 커버리지 증가)")
        else:
            logger.info("   ⚠️ min=2 모델 성능 하락. 커버리지 이득과 Trade-off 고려 필요")
            
else:
    logger.warning("⚠️ 평가를 수행할 데이터나 모델이 부족합니다.")

📊 성능 비교 시작 (샘플 1000개)...


[min3 모델 결과]
   ► Recall@20 Mean : 0.0249 (2.49%)
   ► Recall@20 Median: 0.0000
   ► Cold Start Rate : 8.8%
----------------------------------------
[min2 모델 결과]
   ► Recall@20 Mean : 0.0253 (2.53%)
   ► Recall@20 Median: 0.0000
   ► Cold Start Rate : 4.9%
----------------------------------------
📈 [최종 비교]
   min=2 모델이 min=3 대비 Recall이 +1.6% 변화했습니다. (+0.0004)
   ✅ min=2 모델 채택 권장 (성능 유지/향상 + 커버리지 증가)
